# Visualizing the Philips Curve

The Philips Curve was initially discovered as a statistical relationship between unemployment and inflation. The original version used historical US data.

![Philips Curve](philips_curve.png "Original Philips Curve")

Our goal here is to visually inspect the Philips curve using recent data, for several countries.

In the process we will learn to:
- import dataframes, inspect them, merge them, clean the resulting data
- use matplotlib to create graphs
- bonus: experiment with other plotting libraries

## Importing the Data

We start by loading library dbnomics which contains all the data we want. It is installed already on the nuvolos server.

In [1]:
import dbnomics

The following code imports data for from dbnomics for a few countries.

In [2]:
table_1 = dbnomics.fetch_series([
    "OECD/DP_LIVE/FRA.CPI.TOT.AGRWTH.Q",
    "OECD/DP_LIVE/GBR.CPI.TOT.AGRWTH.Q",
    "OECD/DP_LIVE/USA.CPI.TOT.AGRWTH.Q",
    "OECD/DP_LIVE/DEU.CPI.TOT.AGRWTH.Q"
])

In [3]:
table_2 = dbnomics.fetch_series([
    "OECD/MEI/DEU.LRUNTTTT.STSA.Q",
    "OECD/MEI/FRA.LRUNTTTT.STSA.Q",
    "OECD/MEI/USA.LRUNTTTT.STSA.Q",
    "OECD/MEI/GBR.LRUNTTTT.STSA.Q"
])

__Describe concisely the data that has been imported (periodicity, type of measure, ...). You can either check dbnomics website or look at the databases.__

 table_1 contains quarterly data about growth rate of CPI inflation for several countries (FRA, GBR, USA, DEU).
 There 1084 observations from 1956 to 2023.

In [9]:
table_1.describe()

,period,value
count,1084,1083.000000
mean,1989-09-30 18:46:29.667896704,3.891054
min,1956-01-01 00:00:00,-1.623360
25%,1972-10-01 00:00:00,1.680341
50%,1989-10-01 00:00:00,2.724924
75%,2006-10-01 00:00:00,5.002851
max,2023-07-01 00:00:00,26.565810
std,NaN,3.591647


 table_2 contains quarterly data about unemployment rate from the Labour Force Survey for the same countries from 1956 to 2023.

In [12]:
table_2['series_name'].unique()

array(['Germany – Labour Force Survey - quarterly rates > Unemployment rate > Aged 15 and over > All persons – Level, rate or national currency, s.a. – Quarterly',
       'France – Labour Force Survey - quarterly rates > Unemployment rate > Aged 15 and over > All persons – Level, rate or national currency, s.a. – Quarterly',
       'United States – Labour Force Survey - quarterly rates > Unemployment rate > Aged 15 and over > All persons – Level, rate or national currency, s.a. – Quarterly',
       'United Kingdom – Labour Force Survey - quarterly rates > Unemployment rate > Aged 15 and over > All persons – Level, rate or national currency, s.a. – Quarterly'],
      dtype=object)

In [10]:
table_2.describe()

,period,original_value,value
count,817,817.000000,817.000000
mean,1994-11-08 05:59:33.561811584,6.098550,6.098550
min,1955-01-01 00:00:00,0.373774,0.373774
25%,1979-07-01 00:00:00,4.366667,4.366667
50%,1996-07-01 00:00:00,5.833333,5.833333
75%,2011-01-01 00:00:00,7.996948,7.996948
max,2023-10-01 00:00:00,13.000000,13.000000
std,NaN,2.550613,2.550613


In [8]:
# show onlye the few firs lines:
table_2.head(2)

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,LOCATION,SUBJECT,MEASURE,FREQUENCY,Country,Subject,Measure,Frequency
0,quarterly,OECD,MEI,Main Economic Indicators Publication,DEU.LRUNTTTT.STSA.Q,Germany – Labour Force Survey - quarterly rate...,1962-Q1,1962-01-01,0.442249,0.442249,DEU,LRUNTTTT,STSA,Q,Germany,Labour Force Survey - quarterly rates > Unempl...,"Level, rate or national currency, s.a.",Quarterly
1,quarterly,OECD,MEI,Main Economic Indicators Publication,DEU.LRUNTTTT.STSA.Q,Germany – Labour Force Survey - quarterly rate...,1962-Q2,1962-04-01,0.444882,0.444882,DEU,LRUNTTTT,STSA,Q,Germany,Labour Force Survey - quarterly rates > Unempl...,"Level, rate or national currency, s.a.",Quarterly


__Show the first rows of each database. Make a list of all columns.__

In [13]:
table_1.head(1)

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,Country,Indicator,Subject,Measure,Frequency
0,quarterly,OECD,DP_LIVE,OECD Data Live dataset,FRA.CPI.TOT.AGRWTH.Q,France – Inflation (CPI) – Total – Annual grow...,1956-Q1,1956-01-01,1.746324,1.746324,FRA,CPI,TOT,AGRWTH,Q,France,Inflation (CPI),Total,Annual growth rate (%),Quarterly


__Compute standard statistics for all variables__

In [15]:
table_1.describe()

,period,value
count,1084,1083.000000
mean,1989-09-30 18:46:29.667896704,3.891054
min,1956-01-01 00:00:00,-1.623360
25%,1972-10-01 00:00:00,1.680341
50%,1989-10-01 00:00:00,2.724924
75%,2006-10-01 00:00:00,5.002851
max,2023-07-01 00:00:00,26.565810
std,NaN,3.591647


__Compute averages and standard deviations for unemployment and inflation, per country.__

In [30]:
# option 1: by using pandas boolean selection 


# extract sub dataframe for  France
# table_1.query("Country=='France'")

# this is a vector with True on rows containing France, False otherwise

for country in ['France', 'United Kingdom', 'United States', 'Germany']:

    sel = table_1["Country"]==country
    sdf = table_1[sel]
    m = sdf['value'].mean()
    s = sdf['value'].std()

    print(f" The mean inflation for {country} is {m}. The standard deviation is {s}")


 The mean inflation for France is 4.218004559985239. The standard deviation is 3.85319032016491
 The mean inflation for United Kingdom is 5.003996036162361. The standard deviation is 4.768592756710978
 The mean inflation for United States is 3.678550917822878. The standard deviation is 2.7795046092217035
 The mean inflation for Germany is 2.659118566666667. The standard deviation is 1.8661314171203747


In [27]:
table_1['Country'].unique()

array(['France', 'United Kingdom', 'United States', 'Germany'],
      dtype=object)

In [34]:
# option 2: by using groupby

print("Here are the means")
table_1.groupby('Country')['value'].mean()



Here are the means


Country
France            4.218005
Germany           2.659119
United Kingdom    5.003996
United States     3.678551
Name: value, dtype: float64

In [36]:
print("Here are the standard deviations")
table_1.groupby('Country')['value'].std()


Here are the standard deviations


Country
France            3.853190
Germany           1.866131
United Kingdom    4.768593
United States     2.779505
Name: value, dtype: float64

In [38]:
# To get both mean and std:
table_1.groupby('Country')['value'].agg( ['mean', 'std'] )

,mean,std
Country,,
France,4.218005,3.853190
Germany,2.659119,1.866131
United Kingdom,5.003996,4.768593
United States,3.678551,2.779505


In [33]:
table_2.groupby('Country')['value'].mean()


Country
France            8.680560
Germany           4.989272
United Kingdom    6.705114
United States     5.851087
Name: value, dtype: float64

__The following command merges the two databases together. Explain the role of argument `on`. What happened to the column names?__

In [40]:
table_2.head(1)

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,LOCATION,SUBJECT,MEASURE,FREQUENCY,Country,Subject,Measure,Frequency
0,quarterly,OECD,MEI,Main Economic Indicators Publication,DEU.LRUNTTTT.STSA.Q,Germany – Labour Force Survey - quarterly rate...,1962-Q1,1962-01-01,0.442249,0.442249,DEU,LRUNTTTT,STSA,Q,Germany,Labour Force Survey - quarterly rates > Unempl...,"Level, rate or national currency, s.a.",Quarterly


In [41]:
table = table_1.merge(table_2, on=["period", 'Country']) 

We rename the new names for the sake of clarity and normalize everything with lower cases.

In [47]:
table = table.rename(
    columns={
    'period': 'date',         # because it sounds more natural
    'Country': 'country',
    'value_x': 'inflation',
    'value_y': 'unemployment'
    }
)

__On the merged table, compute at once all the statistics computed before (use `groupby` and `agg`).__

In [50]:
table.groupby('country')[ ['inflation','unemployment'] ].agg(['mean','std'])

inflation           unemployment          
                    mean       std         mean       std
country                                                  
France          1.664349  1.381470     8.680560  0.984035
Germany         2.730136  1.918303     4.989272  3.067199
United Kingdom  5.404707  5.253143     6.705114  2.388539
United States   3.678551  2.779505     5.880812  1.668049

Before we process further, we should tidy the dataframe by keeping only what we need.
- Keep only the columns `date`, `country`, `inflation` and `unemployment`
- Drop all na values
- Make a copy of the result

In [52]:
df = table[['date', 'country', 'inflation', 'unemployment']].dropna()

In [53]:
df = df.copy()
# note: the copy() function is here to avoid keeping references to the original database

In [54]:
df

,date,country,inflation,unemployment
0,2003-01-01,France,2.366263,7.922234
1,2003-04-01,France,1.912854,8.089598
2,2003-07-01,France,1.932270,8.036090
3,2003-10-01,France,2.184437,8.349410
4,2004-01-01,France,1.800087,8.518631
...,...,...,...,...
807,2022-07-01,Germany,7.402639,3.181081
808,2022-10-01,Germany,8.580543,3.059473
809,2023-01-01,Germany,8.236768,2.961556
810,2023-04-01,Germany,6.546894,2.963810


__What is the maximum available interval for each country? How would you proceed to keep only those dates where all datas are available? In the following we keep the resulting "cylindric" database.__

Our DataFrame is now ready for further analysis !


## Plotting using matplotlib

Our goal now consists in plotting inflation against unemployment to see whether a pattern emerges. We will first work on France.

In [ ]:
from matplotlib import pyplot as plt

__Create a database `df_fr` which contains only the data for France.__

__The following command create a line plot for `inflation` against `unemployment`. Can you transform it into a scatterplot ?__

In [ ]:
plt.plot(df_fr['unemployment'], df_fr['inflation']) # missing 'o'

__Expand the above command to make the plot nicer (label, title, grid, ...)__

## Visualizing the regression

The following piece of code regresses `inflation` on `unemployment`.

In [ ]:
from statsmodels.formula import api as sm
model = sm.ols(formula='inflation ~ unemployment', data=df_fr)
result = model.fit()

We can use the resulting model to "predict" inflation from unemployment.

In [ ]:
result.predict(df_fr['unemployment'])

__Store the result in df_fr as a new column `reg_unemployment`__

__Add the regression line to the scatter plot.__

__Now we would like to compare all countries. Can you find a way to represent the data for all of them (all on one graph, using subplots...) ?__

__Any comment on these results?__

## Bonus: Visualizing data using altair

Altair is a visualization library (based on [Vega-lite](https://vega.github.io/vega-lite/)) which offers a different syntax to make plots.

It is well adapted to the exploration phase, as it can operate on a full database (without splitting it like we did for matplotlib). It also provides some data transformation tools like regressions, and ways to add some interactivity.

In [ ]:
import altair as alt

__The following command makes a basic plot from the dataframe `df` which contains all the countries. Can you enhance it by providing a title and encoding information to distinguish the various countries (for instance colors)?__

In [ ]:
chart = alt.Chart(df).mark_point().encode(
    x='unemployment',
    y='inflation',
    # add something here
)
chart

__The following graph plots a regression line, but for all countries, it is rather meaningless. Can you restrict the data to France only?__

In [ ]:
# modify the following code
chart = alt.Chart(df).mark_point().encode(
    x='unemployment',
    y='inflation',
)
chart + chart.transform_regression('unemployment', 'inflation').mark_line()

__One way to visualize data consists in adding some interactivity. Add some title and click on the legend__

In [ ]:
#run first then modify the following code

multi = alt.selection_multi(fields=["country"])

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y('country:N', axis=alt.Axis(orient='right')),
    color=alt.condition(multi, 'country:N', alt.value('lightgray'), legend=None)
).add_selection(multi)

chart_2 = alt.Chart(df).mark_point().encode(
    x='unemployment',
    y='inflation',
    color=alt.condition(multi, 'country:N', alt.value('lightgray')),
    # find a way to separate on the graph data from France and US
)

chart_2 | legend

__Bonus question: in the following graph you can select an interval in the left panel to select some subsample. Can you add the regression line(s) corresponding to the selected data to the last graph?__

In [ ]:
brush = alt.selection_interval(encodings=['x'],)

historical_chart_1 = alt.Chart(df).mark_line().encode(
    x='date',
    y='unemployment',
    color='country'
).add_selection(
    brush
)
historical_chart_2 = alt.Chart(df).mark_line().encode(
    x='date',
    y='inflation',
    color='country'
)
chart = alt.Chart(df).mark_point().encode(
    x='unemployment',
    y='inflation',
    # find a way to separate on the graph data from France and US
    color=alt.condition(brush, 'country:N', alt.value('lightgray'))
)
alt.hconcat(historical_chart_1, historical_chart_2, chart,)

## Bonus 2: Plotly Express

Another popular option is the plotly library for nice-looking interactive plots. Combined with dash or shiny, it can be used to build very powerful interactive interfaces.

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(df, x='unemployment', y='inflation', color='country', title="Philips Curves")
fig

__Improve the graph above in any way you like__